In [ ]:
#适用于zeit.de：得到页面上新闻的链接->跳转->提取文字加入语料库
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import csv
import time

#将raw string转换为normal string
def copeUnicode(text):
    text=text.replace(r'\"','h')
    text=text.replace(r'\u00f6', 'ö')
    text=text.replace(r'\u00e4', 'ä')
    text=text.replace(r'\u00fc', 'ü')
    text=text.replace(r'\u00df', 'ß')
    return text

def writeToCSV(text):
    #先从字符串中提取信息，再写入csv文件
    begin=text.find('"url"')
    end=text.find('"title"')
    url=text[begin+8:end-15]
    begin=text.find('"title"')
    end=text.find('"type"')
    title=text[begin+10:end-15]
    begin=text.find('"tag"')
    end=text.find('"publishtime"')
    tag=text[begin+8:end-15]
    
    with open("newsList.csv","a",newline='') as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerow([copeUnicode(title),copeUnicode(url),copeUnicode(tag)])

def saveInfo(url):
    try:
        #用url得到html
        html = urllib.request.urlopen(url).read()

        soup = BeautifulSoup(html,'html.parser')

        #找到新闻信息
        info = soup.find('script',class_='kilkaya_meta')
        writeToCSV(info.string)

    except:
        print('wrong, try again')      
        saveInfo(url)

#通过主页面链接进入新闻界面，并保存信息
def getNewsPage(url):
    try:
        #用url得到html
        html = urllib.request.urlopen(url).read()

        soup = BeautifulSoup(html,'html.parser')
        #print(soup.prettify())
    
        #tags = soup('link')
        preLink = ''

        tags = soup('a')
        for tag in tags:
            link = tag.get('href') #得到链接
            #排除其他链接，找到新闻页面的链接
            if link is None:
                continue
            if link == preLink: #去除重复链接
                continue        
            if '#comments' in link: #去除指向该文章评论区的链接
                continue          
            if link[:20] != 'https://www.zeit.de/':        
                continue
            if '20' in link: #该网站新闻链接中均有年份，故可根据'20'是否在链接中来判别  
                preLink = link
                saveInfo(link)
                time.sleep(1) #读一个链接休息1秒
    except:
        print('wrong, try again')      
        getNewsPage(url)

def main():
    #准备工作
    f=open('newsList.csv', "r+")
    f.truncate() #清空文件
    f.close()
    with open("newsList.csv","a",newline='') as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerow(['title','url','tag']) #输入表头

    url = 'https://www.zeit.de/suche/index?q=zukunft&type=article&type=gallery&type=video&type=author&type=manualtopic'
    getNewsPage(url)
    for i in range(2,10):
        url = 'https://www.zeit.de/suche/index?q=zukunft&type=article&type=gallery&type=video&type=author&type=manualtopic&p='+str(i+1)
        getNewsPage(url)

main()